# Running a raffle with Marlowe

The raffle contract receives a deposit of a prize and then an oracle randomly selects the winner. The contract would be too large to fit on the blockchain because of the hundreds of possible winners, so Marlowe's merkleization capability is used to break up the contract into manageable pieces. The pieces are "glued together" using Marlowe's `Notify` block, so that the contract can walk a hierarchy of possible winners down to the actual winner.

Below is a diagram of the raffle contract, showing the transactions as arrows:
1.  Create the contract.
2.  Deposit the prize token.
3.  The oracle randomly chooses a winner.
4.  The first notify branches five-fold according the the random number chosen by the oracle.
    -   The *first* branch is taken if the winner is among the *first* group of 125 addresses.
    -   The *second* branch is taken if the winner is among the *second* group of 125 addresses.
    -   The *third* branch is taken if the winner is among the *third* group of 125 addresses.
    -   The *fourth* branch is taken if the winner is among the *fourth* group of 125 addresses.
    -   The *fifth* branch is taken if the winner is among the *fifth* group of 125 addresses.
5.  The second notify branches five-fold according the the random number chosen by the oracle.
    -   The first branch is taken if the winner is among the first group of 25 addresses within the previously selected 125 addresses.
    -   Etc.
6.  The third notify branches five-fold according the the random number chosen by the oracle.
    -   The first branch is taken if the winner is among the first group of 5 addresses within the previously selected 25 addresses.
    -   Etc.
7.  The fourth notify branches five-fold according the the random number chosen by the oracle.
    -   The first branch is taken if the winner is among the first of the previously selected 5 addresses, and the prize is paid to that address.
    -   Etc.
    
![Hierarchy of transactions in the raffle contract.](hierarchy.png)

## Prerequisites

#### First prize

We need to know the policy ID and token name for the NFT serves as the Marlowe role token for redeeming the first prize. There is no requirement for this token beyond that we know its policy ID and token name.

For the purposes of this narrative example, let's say we have the following. ***For the real raffle, these values must be replaced by those for the action first-prize redemption token.***

In [12]:
# Replace with actual policy and name!
PRIZE1_POLICY=8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d
PRIZE1_NAME=1stPrize

#### Second prize

Now repeat the same steps for the second prize.

For the purposes of this narrative example, let's say we have the following. For the real raffle, these values must be replaced by those for the action second-prize redemption token.

In [22]:
# Replace with actual policy and name!
PRIZE2_POLICY=8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d
PRIZE2_NAME=2ndPrize

#### Third prize

Now repeat the same steps for the third prize.

For the purposes of this narrative example, let's say we have the following. For the real raffle, these values must be replaced by those for the action third-prize redemption token.

In [31]:
# Replace with actual policy and name!
PRIZE3_POLICY=8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d
PRIZE3_NAME=3rdPrize

## Running the raffle

### Ensure that the three prize tokens reside at the sponsor address

Send the three prize tokens (the Marlowe role tokens) to the sponsor address.

In [43]:
SPONSOR_SKEY=sponsor.skey
SPONSOR_ADDR=$(cat sponsor.mainnet.address)
echo "SPONSOR_ADDR = $SPONSOR_ADDR"

SPONSOR_ADDR = addr1q95e9feu4hkp4qwvgqasq02na05z3eg33zzjquf2d86e6qzznwng4gtlladnxm7d486psa003jy6dv230t82rvv3pflqeuzzt2


After the tokens have been sent, we can query to see them.

In [44]:
cardano-cli query utxo --mainnet --address $SPONSOR_ADDR

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
63fe0d17bfdc9749c64858abcb9111f448ddbedd28a3a8e45a076faefadc0e06     0        38203831 lovelace + TxOutDatumNone
63fe0d17bfdc9749c64858abcb9111f448ddbedd28a3a8e45a076faefadc0e06     1        2000000 lovelace + 1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.3173745072697a65 + TxOutDatumNone
63fe0d17bfdc9749c64858abcb9111f448ddbedd28a3a8e45a076faefadc0e06     2        2000000 lovelace + 1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.326e645072697a65 + TxOutDatumNone
63fe0d17bfdc9749c64858abcb9111f448ddbedd28a3a8e45a076faefadc0e06     3        2000000 lovelace + 1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.3372645072697a65 + TxOutDatumNone


### Download the executables to run the raffle.

The script residing at https://github.com/input-output-hk/real-world-marlowe/tree/main/archives/raffle are compiled to executables that should work on any recent Linux machine.

In [45]:
curl -sS http://dl.marlowestat.org/raffle-bin.tar -o raffle-bin.tar

Unpack the executable files.

In [46]:
tar xvf raffle-bin.tar

bin/
bin/cardano-cli
bin/marlowe-cli
bin/marlowe-runtime-cli
bin/InitializeRaffle
bin/ExecuteRaffle


Put the executables at the front of the search path.

In [47]:
export PATH=$PWD/bin:$PATH

### Run the raffle for the first prize

First create the file that describes the role token holding the first prize.

In [48]:
cat << EOI > first-prize.token
[
    [
        "$PRIZE1_POLICY",
        "$PRIZE1_NAME"
    ]
]
EOI
cat first-prize.token

[
    [
        "8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d",
        "1stPrize"
    ]
]


Now create the file with the addresses of the raffle ticket holders.

In [49]:
head first-prize.addresses

[{"asset_name":"5469636b6574233939","payment_address":"addr1q97q4e9nlhq08mrugs8pm5uq045e425w72mxhxupq9qqgykr7cz4mu6gh005gdck67p7y9d8s8zsfgjkcdy75mrjh6jq3dk3ng","quantity":"1"},
 {"asset_name":"5469636b6574233938","payment_address":"addr1qxmph7666hj5yadja34pdlm8n5un0fhc9p0hv2g9ask7m49n3kr657fpa3q8mzwmjqvl9lqdn9g2pm3ejhlgwpprwy2sq5s22j","quantity":"1"},
 {"asset_name":"5469636b6574233937","payment_address":"addr1q8pdjzhjthflmtu947feruy6rnu0zr8x7xlppgqxrew62xqwpjujxwjglw2gelk9thqcta802lprejcu4y8dtz64uhnsaeaet6","quantity":"1"},
 {"asset_name":"5469636b6574233936","payment_address":"addr1q9zun0s6thsyf9qt0zyfnn50x8kpwutrv20eq8dgx9gt6tkr7cz4mu6gh005gdck67p7y9d8s8zsfgjkcdy75mrjh6jq2x0dua","quantity":"1"},
 {"asset_name":"5469636b6574233935","payment_address":"addr1qy5sj3v265ucpuy7ea7h7qy533xnhcl9ru3g24xcetrnxu4n3kr657fpa3q8mzwmjqvl9lqdn9g2pm3ejhlgwpprwy2slm3vfx","quantity":"1"},
 {"asset_name":"5469636b6574233934","payment_address":"addr1qy5lsaapah8d5pf8s2uvnnas4yk9s55vgns4jsvzz5exksgwpjujxwj

Now create the configuration file for running the raffle.

In [50]:
yaml2json << EOI > first-prize.config
chunkSize: 5          # Danger: do not change the chunk size.
deadlines:
  deposit: 3d         # Deadline for depositing the prize token.
  selectWinner: 3d    # Deadline for selecting the winner.
  payout: 3d          # Deadline for paying the winner.
runtimeURI:
  proxy_host: 34.70.189.123  # The IP address of Marlowe Runtime v0.0.4. Use $MARLOWE_RT_HOST if a backup is needed.
  proxy_port: 3700           # The port number for Marlowe Runtime's proxy service. Use $MARLOWE_RT_PORT for the backup.
  web_host: 34.70.189.123    # The IP address of Marlowe Runtime v0.0.4. Use $MARLOWE_RT_WEBSERVER_HOST if a backup is needed.
  web_port: 3780             # The port number for Marlowe Runtime's web server. Use $MARLOWE_RT_WEBSERVER_PORT for the backup.
sponsorAddressFilePath: sponsor.mainnet.address
sponsorCollateralFilePath: not-used
sponsorPrivateKeyFilePath: sponsor.skey
contract: first-prize.contract
state: first-prize.state
tmpTxToSign: temp.unsigned
tmpTxToSubmit: temp.signed
EOI
cat first-prize.config

{"chunkSize":5,"contract":"first-prize.contract","deadlines":{"deposit":"3d","payout":"3d","selectWinner":"3d"},"runtimeURI":{"host":"34.70.189.123","proxy_port":3700,"web_port":3780},"sponsorAddressFilePath":"sponsor.mainnet.address","sponsorCollateralFilePath":"not-used","sponsorPrivateKeyFilePath":"sponsor.skey","state":"first-prize.state","tmpTxToSign":"temp.unsigned","tmpTxToSubmit":"temp.signed"}


Now build and submit the transaction that initializes the raffle.

In [51]:
InitializeRaffle first-prize.config first-prize.addresses first-prize.token

#########################
Raffle Contract Generation & Initialisation
-------------------------
 >> Raffle Contract saved in : first-prize.contract
-------------------------
Raffle Contract Generated
-------------------------
 >> Contract stored with hash: 79316f8ec81177f8797ef2e953b52773f9007f9c9c4f4c681ef5f259fcf190dc
Safety analysis found the following errors in the contract:
- detail: The safety analysis exceeded the allotted time.
  error: SafetyAnalysisTimeout
  fatal: false

{"contractId":"9815e1a9b2bed549d60bd0b903166f79958888f727853be8690f84554e308d1d#1"}
 >> tx signed
{"blockHeaderHash":"36b1047cffbf12c888bec87ce5fb1b12ecf1fb31c42c9bf58e335e23f60dd8bb","blockNo":9190057,"slotNo":101092657}
 >> tx submitted
 >> Contract initialzed (tx appended)
-------------------------
Raffle Contract Initialized : {"contractId":"9815e1a9b2bed549d60bd0b903166f79958888f727853be8690f84554e308d1d#1"}
-------------------------
#########################
{"contractId":"9815e1a9b2bed549d60bd0b903166

Record the contract ID that was printed.

In [52]:
PRIZE1_CONTRACT=9815e1a9b2bed549d60bd0b903166f79958888f727853be8690f84554e308d1d#1

Run the raffle itself, which consists of several transactions:
1. Deposit the prize token.
2. Have the oracle retrieve a random number from https://www.random.org/.
3. Execute several Marlowe notify transactions to locate the winner.
4. Send the prize token to the winner.

In [53]:
ExecuteRaffle first-prize.config first-prize.addresses first-prize.token $PRIZE1_CONTRACT


#########################
WaitingForPrizeDeposit
 >> Depositing 1stPrize
{"txId":"72c6ec4b45a0490ec6d99b6f752a5ff663db459c6963ff5fe9efa0655f5812c2"}
 >> tx signed
{"blockHeaderHash":"95f80c8af22747489a34f55d3286683731f1ccb68287513070f408fd66368b97","blockNo":9190063,"slotNo":101092712}
 >> tx submitted
 >> 1stPrize deposited
>>>> Prize NFTs Deposited
#########################

#########################
WaitingForOracle
#########################
{"txId":"7657ab07d74ff11d50be28111dae82ebac77ae71fbf3c6d6e37d9d178a7f75af"}
 >> tx signed
{"blockHeaderHash":"b8e67b13c7fa6961e92689184c20549d83be0d0630dac74f52a2a1a50f0d4ad2","blockNo":9190064,"slotNo":101092729}
 >> tx submitted
>>>> Oracle has answered with 268
>>>> Winning ticket is 5469636b657423333938 (Ticket#398)
>>>> Snapshotted address is addr1qxe3seuxx0kdylmr26an7hh4xsn8z4jkk7q9ha3lun6w08dn3kr657fpa3q8mzwmjqvl9lqdn9g2pm3ejhlgwpprwy2sr0avw6
#########################

#########################
WaitingForNotify
#########################


View the contract on MarloweScan.

In [54]:
echo "https://mainnet.marlowescan.com/contractView?tab=tx-list&contractId=${PRIZE1_CONTRACT%%#1}%231"

https://mainnet.marlowescan.com/contractView?tab=tx-list&contractId=9815e1a9b2bed549d60bd0b903166f79958888f727853be8690f84554e308d1d%231


Record the random number from that the oracle choose, listed in the output above.

In [55]:
PRIZE1_CHOICE=268

Compute the address of the winner and view their prize on an explorer.

In [56]:
PRIZE1_WINNER=$(jq -r ".[$PRIZE1_CHOICE].payment_address" first-prize.addresses)
echo "PRIZE1_WINNER = $PRIZE1_WINNER"
echo "https://cardanoscan.io/address/$PRIZE1_WINNER"

PRIZE1_WINNER = addr1qxe3seuxx0kdylmr26an7hh4xsn8z4jkk7q9ha3lun6w08dn3kr657fpa3q8mzwmjqvl9lqdn9g2pm3ejhlgwpprwy2sr0avw6
https://cardanoscan.io/address/addr1qxe3seuxx0kdylmr26an7hh4xsn8z4jkk7q9ha3lun6w08dn3kr657fpa3q8mzwmjqvl9lqdn9g2pm3ejhlgwpprwy2sr0avw6


### Run the raffle for the second prize

First create the file that describes the role token holding the second prize.

In [57]:
cat << EOI > second-prize.token
[
    [
        "$PRIZE2_POLICY",
        "$PRIZE2_NAME"
    ]
]
EOI
cat second-prize.token

[
    [
        "8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d",
        "2ndPrize"
    ]
]


Now create the file for those eligible for the second prize by deleting the entry for the first prize.

In [58]:
jq "del(.[$PRIZE1_CHOICE])" first-prize.addresses > second-prize.addresses

Now create the configuration file for running the raffle.

In [59]:
yaml2json << EOI > second-prize.config
chunkSize: 5          # Danger: do not change the chunk size.
deadlines:
  deposit: 3d         # Deadline for depositing the prize token.
  selectWinner: 3d    # Deadline for selecting the winner.
  payout: 3d          # Deadline for paying the winner.
runtimeURI:
  proxy_host: 34.70.189.123  # The IP address of Marlowe Runtime v0.0.4. Use $MARLOWE_RT_HOST if a backup is needed.
  proxy_port: 3700           # The port number for Marlowe Runtime's proxy service. Use $MARLOWE_RT_PORT for the backup.
  web_host: 34.70.189.123    # The IP address of Marlowe Runtime v0.0.4. Use $MARLOWE_RT_WEBSERVER_HOST if a backup is needed.
  web_port: 3780             # The port number for Marlowe Runtime's web server. Use $MARLOWE_RT_WEBSERVER_PORT for the backup.
sponsorAddressFilePath: sponsor.mainnet.address
sponsorCollateralFilePath: not-used
sponsorPrivateKeyFilePath: sponsor.skey
contract: second-prize.contract
state: second-prize.state
tmpTxToSign: temp.unsigned
tmpTxToSubmit: temp.signed
EOI
cat second-prize.config

{"chunkSize":5,"contract":"second-prize.contract","deadlines":{"deposit":"3d","payout":"3d","selectWinner":"3d"},"runtimeURI":{"host":"34.70.189.123","proxy_port":3700,"web_port":3780},"sponsorAddressFilePath":"sponsor.mainnet.address","sponsorCollateralFilePath":"not-used","sponsorPrivateKeyFilePath":"sponsor.skey","state":"second-prize.state","tmpTxToSign":"temp.unsigned","tmpTxToSubmit":"temp.signed"}


Now build and submit the transaction that initializes the raffle.

In [60]:
InitializeRaffle second-prize.config second-prize.addresses second-prize.token

#########################
Raffle Contract Generation & Initialisation
-------------------------
 >> Raffle Contract saved in : second-prize.contract
-------------------------
Raffle Contract Generated
-------------------------
 >> Contract stored with hash: 428cade8bc26e876771c853ef51a00b629f2f342f2566a509ea4e08ed4a409a8
Safety analysis found the following errors in the contract:
- detail: The safety analysis exceeded the allotted time.
  error: SafetyAnalysisTimeout
  fatal: false

{"contractId":"03af66e2a7555055ba5a3ef0e33fe7b54d28c6dc0cdbf0e58fb359a677faa064#1"}
 >> tx signed
{"blockHeaderHash":"b5024f915888c5f202096e51ac2234722633099bfa1f660c501512e2be8bbd9c","blockNo":9190182,"slotNo":101095005}
 >> tx submitted
 >> Contract initialzed (tx appended)
-------------------------
Raffle Contract Initialized : {"contractId":"03af66e2a7555055ba5a3ef0e33fe7b54d28c6dc0cdbf0e58fb359a677faa064#1"}
-------------------------
#########################
{"contractId":"03af66e2a7555055ba5a3ef0e33f

Record the contract ID that was printed.

In [61]:
PRIZE2_CONTRACT=03af66e2a7555055ba5a3ef0e33fe7b54d28c6dc0cdbf0e58fb359a677faa064#1

Run the raffle itself, which consists of several transactions.

In [62]:
ExecuteRaffle second-prize.config second-prize.addresses second-prize.token $PRIZE2_CONTRACT


#########################
WaitingForPrizeDeposit
 >> Depositing 2ndPrize
{"txId":"50124e1cf2e566a4a0e2b92230df36500083ef7cca009e19b35859596890fc0f"}
 >> tx signed
{"blockHeaderHash":"777eb3565f165a9c4eacb4db88499b043911e17eeef0e4b5d2d9c8b7c3fdd6f8","blockNo":9190187,"slotNo":101095125}
 >> tx submitted
 >> 2ndPrize deposited
>>>> Prize NFTs Deposited
#########################

#########################
WaitingForOracle
#########################
{"txId":"809d73f273786ffa76e3f81e976919234694f28fae5942ec58873689b8d8f5b4"}
 >> tx signed
{"blockHeaderHash":"161e94c6c4d446b7fa47f0659780c53420e6e56ec6f54f599aed26c40fe34593","blockNo":9190188,"slotNo":101095141}
 >> tx submitted
>>>> Oracle has answered with 180
>>>> Winning ticket is 5469636b657423343737 (Ticket#477)
>>>> Snapshotted address is addr1q9flz39mjz7pqafjmrrstctdu6ttls3wnhm08y3l7q8cq3wr7cz4mu6gh005gdck67p7y9d8s8zsfgjkcdy75mrjh6jqmrx7sg
#########################

#########################
WaitingForNotify
#########################


View the contract on MarloweScan.

In [63]:
echo "https://mainnet.marlowescan.com/contractView?tab=tx-list&contractId=${PRIZE2_CONTRACT%%#1}%231"

https://mainnet.marlowescan.com/contractView?tab=tx-list&contractId=03af66e2a7555055ba5a3ef0e33fe7b54d28c6dc0cdbf0e58fb359a677faa064%231


Record the random number from that the oracle choose, listed in the output above.

In [64]:
PRIZE2_CHOICE=180

Compute the address of the winner and view their prize on an explorer.

In [65]:
PRIZE2_WINNER=$(jq -r ".[$PRIZE2_CHOICE].payment_address" second-prize.addresses)
echo "PRIZE2_WINNER = $PRIZE2_WINNER"
echo "https://cardanoscan.io/address/$PRIZE2_WINNER"

PRIZE2_WINNER = addr1q9flz39mjz7pqafjmrrstctdu6ttls3wnhm08y3l7q8cq3wr7cz4mu6gh005gdck67p7y9d8s8zsfgjkcdy75mrjh6jqmrx7sg
https://cardanoscan.io/address/addr1q9flz39mjz7pqafjmrrstctdu6ttls3wnhm08y3l7q8cq3wr7cz4mu6gh005gdck67p7y9d8s8zsfgjkcdy75mrjh6jqmrx7sg


### Run the raffle for the thrid prize

First create the file that describes the role token holding the third prize.

In [66]:
cat << EOI > third-prize.token
[
    [
        "$PRIZE3_POLICY",
        "$PRIZE3_NAME"
    ]
]
EOI
cat third-prize.token

[
    [
        "8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d",
        "3rdPrize"
    ]
]


Now create the file for those eligible for the second prize by deleting the entry for the second prize.

In [67]:
jq "del(.[$PRIZE2_CHOICE])" second-prize.addresses > third-prize.addresses

Now create the configuration file for running the raffle.

In [68]:
yaml2json << EOI > third-prize.config
chunkSize: 5          # Danger: do not change the chunk size.
deadlines:
  deposit: 3d         # Deadline for depositing the prize token.
  selectWinner: 3d    # Deadline for selecting the winner.
  payout: 3d          # Deadline for paying the winner.
runtimeURI:
  proxy_host: 34.70.189.123  # The IP address of Marlowe Runtime v0.0.4. Use $MARLOWE_RT_HOST if a backup is needed.
  proxy_port: 3700           # The port number for Marlowe Runtime's proxy service. Use $MARLOWE_RT_PORT for the backup.
  web_host: 34.70.189.123    # The IP address of Marlowe Runtime v0.0.4. Use $MARLOWE_RT_WEBSERVER_HOST if a backup is needed.
  web_port: 3780             # The port number for Marlowe Runtime's web server. Use $MARLOWE_RT_WEBSERVER_PORT for the backup.
sponsorAddressFilePath: sponsor.mainnet.address
sponsorCollateralFilePath: not-used
sponsorPrivateKeyFilePath: sponsor.skey
contract: third-prize.contract
state: third-prize.state
tmpTxToSign: temp.unsigned
tmpTxToSubmit: temp.signed
EOI
cat third-prize.config

{"chunkSize":5,"contract":"third-prize.contract","deadlines":{"deposit":"3d","payout":"3d","selectWinner":"3d"},"runtimeURI":{"host":"34.70.189.123","proxy_port":3700,"web_port":3780},"sponsorAddressFilePath":"sponsor.mainnet.address","sponsorCollateralFilePath":"not-used","sponsorPrivateKeyFilePath":"sponsor.skey","state":"third-prize.state","tmpTxToSign":"temp.unsigned","tmpTxToSubmit":"temp.signed"}


Now build and submit the transaction that initializes the raffle.

In [69]:
InitializeRaffle third-prize.config third-prize.addresses third-prize.token

#########################
Raffle Contract Generation & Initialisation
-------------------------
 >> Raffle Contract saved in : third-prize.contract
-------------------------
Raffle Contract Generated
-------------------------
 >> Contract stored with hash: 3d91c0acbcf577016754daaf79c42e7aa02b04b177bd5c568573373b0bf468c3
Safety analysis found the following errors in the contract:
- detail: The safety analysis exceeded the allotted time.
  error: SafetyAnalysisTimeout
  fatal: false

{"contractId":"6f13134c0d76c66ca23628cfef1b18bf26f05c7980998af926cb1673795b5dd2#1"}
 >> tx signed
{"blockHeaderHash":"6c3a7444b9cf43c666de16d3fe12a05053c3ca325fb195762dd50f17e4cea938","blockNo":9190202,"slotNo":101095430}
 >> tx submitted
 >> Contract initialzed (tx appended)
-------------------------
Raffle Contract Initialized : {"contractId":"6f13134c0d76c66ca23628cfef1b18bf26f05c7980998af926cb1673795b5dd2#1"}
-------------------------
#########################
{"contractId":"6f13134c0d76c66ca23628cfef1b1

Record the contract ID that was printed.

In [70]:
PRIZE3_CONTRACT=6f13134c0d76c66ca23628cfef1b18bf26f05c7980998af926cb1673795b5dd2#1

Run the raffle itself, which consists of several transactions.

In [71]:
ExecuteRaffle third-prize.config third-prize.addresses third-prize.token $PRIZE3_CONTRACT


#########################
WaitingForPrizeDeposit
 >> Depositing 3rdPrize
{"txId":"397a5f5c6a844fe8927cc34da34cc723140382b1c2f68bc6ce0a0f1d1c5cd016"}
 >> tx signed
{"blockHeaderHash":"67b9c51d47f0a363729894e2f9dcfea933c522778c30a16eb51192b315df43ef","blockNo":9190205,"slotNo":101095492}
 >> tx submitted
 >> 3rdPrize deposited
>>>> Prize NFTs Deposited
#########################

#########################
WaitingForOracle
#########################
{"txId":"94c816b355c993138e71e62b995d1e6e8ba6cb87fd2e164b184aa6634335a4c4"}
 >> tx signed
{"blockHeaderHash":"1eaa3ca0d47687d031d4ee8fcd7897e8792fadda1851bbe96a48f4ab8d8d93ff","blockNo":9190206,"slotNo":101095505}
 >> tx submitted
>>>> Oracle has answered with 70
>>>> Winning ticket is 5469636b657423353736 (Ticket#576)
>>>> Snapshotted address is addr1q8trv6h2t62prkw5kgt2et62fy068ed0u055glrednxwqm9n3kr657fpa3q8mzwmjqvl9lqdn9g2pm3ejhlgwpprwy2sll9f6w
#########################

#########################
WaitingForNotify
#########################
{

View the contract on MarloweScan.

In [72]:
echo "https://mainnet.marlowescan.com/contractView?tab=tx-list&contractId=${PRIZE3_CONTRACT%%#1}%231"

https://mainnet.marlowescan.com/contractView?tab=tx-list&contractId=6f13134c0d76c66ca23628cfef1b18bf26f05c7980998af926cb1673795b5dd2%231


Record the random number from that the oracle choose, listed in the output above.

In [73]:
PRIZE3_CHOICE=70

Compute the address of the winner and view their prize on an explorer.

In [74]:
PRIZE3_WINNER=$(jq -r ".[$PRIZE3_CHOICE].payment_address" third-prize.addresses)
echo "PRIZE3_WINNER = $PRIZE3_WINNER"
echo "https://cardanoscan.io/address/$PRIZE3_WINNER"

PRIZE3_WINNER = addr1q8trv6h2t62prkw5kgt2et62fy068ed0u055glrednxwqm9n3kr657fpa3q8mzwmjqvl9lqdn9g2pm3ejhlgwpprwy2sll9f6w
https://cardanoscan.io/address/addr1q8trv6h2t62prkw5kgt2et62fy068ed0u055glrednxwqm9n3kr657fpa3q8mzwmjqvl9lqdn9g2pm3ejhlgwpprwy2sll9f6w
